In [ ]:
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
%run Attention.py

In [ ]:
Qc_Dataset = torch.load('Qc_3680_2070.pt')
Qc_Dataset.shape

In [ ]:
Qctrain = torch.narrow(Qc_Dataset, 0, 0, 3680)
Qctest = torch.narrow(Qc_Dataset, 0, 3680, 2070)

In [ ]:
class FC_Dataset_partial(Dataset):
    def __init__(self, dataset, n1=300, n2 = 440):
        self.dataset = dataset
        self.n1 = n1
        self.n2 = n2

    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, idx):
        x = self.dataset[:, :self.n1]
        y = self.dataset[:, self.n2:]
        x = x.__getitem__(idx)
        y = y.__getitem__(idx)
        return x, y

In [ ]:
train_data, null_data = torch.utils.data.random_split(FC_Dataset_partial(dataset=Qctrain,n1=400, n2 = 440), (Qctrain.shape[0], 0)) 
null_data, test_data = torch.utils.data.random_split(FC_Dataset_partial(dataset=Qctest,n1=400, n2 = 440), (0, Qctest.shape[0])) 

In [ ]:
# Loading the data through DataLoader:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
# Defining a neural network:

num_features = train_data[0][0].shape[0]
num_targets = train_data[0][1].shape[0]
num_neurons = 512

activ = 'ReLU'

# del fc_model_1
fc_model_1 = nn.Sequential(
    getLayer(num_features, num_neurons, activation=activ),
    getLayer(num_neurons, num_neurons, activation=activ),
    getLayer(num_neurons, 2*num_neurons, activation=activ),
    getLayer(2*num_neurons, 2*num_neurons, activation=activ),
    getLayer(2*num_neurons, 4*num_neurons, activation=activ),
    getLayer(4*num_neurons, 4*num_neurons, activation=activ),
    nn.Linear(4*num_neurons, num_targets),
)

eta = 0.001
loss_func = nn.MSELoss()
optimizer = torch.optim.AdamW(fc_model_1.parameters(), lr=eta)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, gamma=0.3)

In [ ]:
num_features, num_targets

In [ ]:
# Execute function
mem_report()

In [ ]:
# training the fully connected model
start = time.time()
fc_1_results = train_network_reg(fc_model_1, loss_func, train_loader, test_loader=test_loader, epochs=100, score_funcs={'R^2 score': r2_score}, 
                                device=device, optimizer=optimizer, lr_schedule=scheduler)
stop = time.time()
print('Processing time: %s sec' %(stop-start))

In [ ]:
# Plotting the accuracy of the fully connected model:
sns.lineplot(x='epoch', y='test R^2 score', data=fc_1_results, label='fc-1')
plt.title('Test R^2 Score of the First fc-Model')
plt.gcf().set_size_inches(9, 5.5)
plt.show()

In [ ]:
fc_1_results